# Auteurs : BAUDET Quentin & LARMAILLARD-NOIREN Joris

Avant de construire notre modèle de recommandation de smartphones, il est crucial de bien comprendre les caractéristiques et la structure du dataset fourni. Dans ce notebook, nous réalisons une Analyse Exploratoire des Données (EDA) afin de mieux cerner la distribution des différentes variables, détecter les éventuelles anomalies, valeurs manquantes ou incohérences, et identifier les relations potentielles entre les attributs.

Cette étape nous permettra d’extraire des insights précieux pour orienter le pré-traitement des données et la sélection des fonctionnalités les plus pertinentes, en vue d’optimiser les performances du modèle de recommandation.

### Chargement des données

In [1]:
### Importation des modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
### Chargement des données
data = pd.read_csv('../data/raw/Smartphones_cleaned_dataset.csv')

data.head()

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,refresh_rate,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,resolution_width,resolution_height
0,oneplus,OnePlus 11 5G,54999,89.0,True,True,False,snapdragon,8.0,3.2,...,120,3,1.0,android,50.0,16.0,0,NaN,1440,3216
1,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,True,False,False,snapdragon,8.0,2.2,...,120,3,1.0,android,64.0,16.0,1,1024.0,1080,2412
2,samsung,Samsung Galaxy A14 5G,16499,75.0,True,False,False,exynos,8.0,2.4,...,90,3,1.0,android,50.0,13.0,1,1024.0,1080,2408
3,motorola,Motorola Moto G62 5G,14999,81.0,True,False,False,snapdragon,8.0,2.2,...,120,3,1.0,android,50.0,16.0,1,1024.0,1080,2400
4,realme,Realme 10 Pro Plus,24999,82.0,True,False,False,dimensity,8.0,2.6,...,120,3,1.0,android,108.0,16.0,0,NaN,1080,2412


Dans un premier temps nous allons reformater les prix en euros : les données qui sont présentées ci-dessus viennent du marché des smartphones en Inde. Nous allons donc faire une conversion des prix, ici en roupie, en euros.

In [3]:
### Conversion des prix roupie -> euro
data['price'] = data['price'] * 0.009966

### Arrondi des prix
data['price'] = data['price'].round(2)

data = data.rename(columns={'price': 'price (€)'})

data.head()

,brand_name,model,price (€),rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,refresh_rate,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,resolution_width,resolution_height
0,oneplus,OnePlus 11 5G,548.12,89.0,True,True,False,snapdragon,8.0,3.2,...,120,3,1.0,android,50.0,16.0,0,NaN,1440,3216
1,oneplus,OnePlus Nord CE 2 Lite 5G,199.21,81.0,True,False,False,snapdragon,8.0,2.2,...,120,3,1.0,android,64.0,16.0,1,1024.0,1080,2412
2,samsung,Samsung Galaxy A14 5G,164.43,75.0,True,False,False,exynos,8.0,2.4,...,90,3,1.0,android,50.0,13.0,1,1024.0,1080,2408
3,motorola,Motorola Moto G62 5G,149.48,81.0,True,False,False,snapdragon,8.0,2.2,...,120,3,1.0,android,50.0,16.0,1,1024.0,1080,2400
4,realme,Realme 10 Pro Plus,249.14,82.0,True,False,False,dimensity,8.0,2.6,...,120,3,1.0,android,108.0,16.0,0,NaN,1080,2412


**Dimension du dataset**

In [4]:
data.shape

(980, 26)

**Colonnes du dataset**

In [10]:
data.columns

Index(['brand_name', 'model', 'price (€)', 'rating', 'has_5g', 'has_nfc',
       'has_ir_blaster', 'processor_brand', 'num_cores', 'processor_speed',
       'battery_capacity', 'fast_charging_available', 'fast_charging',
       'ram_capacity', 'internal_memory', 'screen_size', 'refresh_rate',
       'num_rear_cameras', 'num_front_cameras', 'os', 'primary_camera_rear',
       'primary_camera_front', 'extended_memory_available', 'extended_upto',
       'resolution_width', 'resolution_height'],
      dtype='object')

Maintenant, nous allons passer par l'étape d'exploration des données.

### EDA

In [5]:
### Importation des modules
import plotly.express as px
import plotly.graph_objects as go

**Colonne des marques**

In [6]:
### Distribution des marques
fig_brand = px.histogram(data, x='brand_name', nbins=20, title="Distribution des marques de téléphones vendus")

fig_brand.update_layout(
    xaxis_title='Marque',
    yaxis_title='Nombre de smartphones',
    bargap=0.1
)

fig_brand.show()

* Marques les plus fréquentes :
    - Xiaomi, OnePlus, et Samsung sont les plus représentées, avec environ 130 modèles ou variantes chacun.
    - Apple, Realme, Motorola, Infinix, Vivo, Oppo et Nokia suivent avec une forte présence.
* Longue traîne de petites marques :
    - De nombreuses marques (plus de 20) sont représentées de façon marginale (moins de 10 modèles), voire parfois très peu présentes.


**Colonnes des prix**

Ici, nous affichons la distribution des prix des smartphones.

In [7]:
### Distribution des prix
fig_price = px.histogram(data, x='price (€)', nbins=20, title='Distribution des prix des smartphones')

fig_price.update_layout(
    xaxis_title='Prix (€)',
    yaxis_title='Nombre de smartphones',
    bargap=0.1
)

fig_price.show()

* Distribution très asymétrique : la majorité écrasante des smartphones sont concentrés dans la tranche basse des prix, en dessous de 1 000 €.
* Longue queue vers la droite : quelques smartphones très haut de gamme (jusqu’à 6000 €) tirent la distribution vers la droite, mais ils sont extrêmement rares.
* Pointe forte autour de l'entrée de gamme : le pic de fréquence se situe dans les premiers intervalles de prix (probablement entre 100 € et 500 €).

**Prix vs. Capacité de la batterie**

In [8]:
fig_battery_price = px.scatter(
    data,
    x='battery_capacity',
    y='price (€)',
    color='brand_name',
    title='Prix vs Capacité de batterie des smartphones',
    labels={'battery': 'Batterie (mAh)', 'price': 'Prix (€)'},
    opacity=0.6
)

fig_battery_price.update_layout(
    xaxis_title='Capacité batterie',
    yaxis_title='Prix (€)',
    bargap=0.1
)

fig_battery_price.show()

* Tendance générale :
    - Il n’y a pas de corrélation claire entre le prix et la capacité de batterie.
    - Des téléphones à prix très bas peuvent avoir une batterie très élevée (outliers visibles à droite).
    - Inversement, certains téléphones chers n'ont qu'une capacité moyenne.
* Présence d’outliers :
    - Des smartphones affichent une capacité batterie supérieure à 10 000 mAh, ce qui est anormalement élevé pour des téléphones classiques → probablement des erreurs ou des modèles atypiques (tablettes, rugged phones).
* Dispersion importante :
    - Il y a une forte variabilité des prix pour une même capacité batterie, ce qui montre que la batterie n’est pas un facteur déterminant du prix.

**Comparaison entre le prix et les différentes colonnes**

In [9]:
### Liste des colonnes à comparer au prix
colonnes = ['rating', 'num_cores', 'processor_speed', 'ram_capacity', 'internal_memory', 'screen_size', 'refresh_rate', ]

### Création des traces pour chaque colonne
fig_compare_to_price = go.Figure()

for i, col in enumerate(colonnes):
    fig_compare_to_price.add_trace(
        go.Scatter(
            x=data[col],
            y=data['price (€)'],
            mode='markers',
            name=col,
            visible=(i == 0)
        )
    )

### Création du menu déroulant
menu = [dict(
    buttons=[dict(
        label=col,
        method='update',
        args=[{'visible': [i == j for j in range(len(colonnes))]},
              {'xaxis': {'title': col}}]
    ) for i, col in enumerate(colonnes)],
    direction='down',
    showactive=True,
    x=0.1,
    y=1.15
)]

fig_compare_to_price.update_layout(
    updatemenus=menu,
    title='Prix vs variable sélectionnée',
    xaxis_title=colonnes[0],
    yaxis_title='Prix (€)',
    height=600
)

fig_compare_to_price.show()

**Rating vs prix**

* On observe une légère tendance croissante : les téléphones avec un meilleur rating tendent à être plus chers, surtout à partir de 75+.
* Cela peut s’expliquer par le fait que les modèles mieux notés sont généralement plus performants ou mieux finis.
* La majorité des téléphones ont une note comprise entre 75 et 90.
* Très peu de modèles en dessous de 70, ce qui peut indiquer un biais dans les avis (seuls les bons modèles sont évalués ou conservés dans le dataset).

**Processor_speed vs prix**

* On observe une corrélation positive modérée : en général, plus la vitesse du processeur augmente, plus les smartphones ont tendance à être chers.
* Beaucoup de modèles se situent entre 2.0 et 3.2 GHz, avec des prix allant de 100 € à 1000 €.
* Cela semble représenter le segment "standard à performant" du marché.
* Plus on monte en vitesse, plus les prix deviennent variables, indiquant que la vitesse du processeur n’est pas le seul facteur déterminant du prix.

**Internal_memory vs prix**

* Il existe une relation positive globale entre la capacité de stockage (en Go) et le prix du smartphone.
* On voit des groupes bien distincts :
    - 32–64 Go : téléphones d’entrée de gamme à prix très accessibles.
    - 128–256 Go : majorité des modèles milieu/haut de gamme.
    - 512–1024 Go (1 To) : modèles premium uniquement.
* À 256 Go, par exemple, les prix vont de ~400 € à plus de 2000 € → cela indique que le stockage ne suffit pas à expliquer le prix à lui seul.
* D’autres caractéristiques comme la RAM, le processeur, la marque ou la caméra jouent un rôle majeur.
* 

**Screen_size vs prix**

* Il existe une légère corrélation positive entre la taille de l’écran et le prix : les téléphones avec un écran plus grand sont en moyenne un peu plus chers.
* Très forte densité autour de 6.3 à 6.7 pouces, avec des prix allant de 100 € à 1500 €.
* Cela suggère que la plupart des smartphones sont conçus pour offrir un compromis entre confort d’écran et ergonomie.
* À taille d’écran égale, le prix peut énormément varier → ce qui confirme que l’écran est un critère secondaire dans la tarification par rapport à :
    - la marque,
    - le processeur,
    - les fonctionnalités avancées (caméras, design, etc.).

**Refresh_rate vs prix**

* On observe une élévation du prix moyen à mesure que le taux de rafraîchissement augmente.
    - 60 Hz → présent sur des modèles de toutes gammes (entrée à premium).
    - 120 Hz et + → associés à des smartphones plus récents ou gaming, donc souvent plus chers.
* À 60 Hz, très grande dispersion de prix (de < 100 € à > 6000 €), ce qui confirme que ce n’est pas un critère déterminant seul.
* À 120–144 Hz, les prix restent modérément élevés avec moins de dispersion : cette technologie est réservée aux gammes moyennes à hautes.
* À 240 Hz, très peu de modèles → probablement des modèles gaming haut de gamme.